# Compute CRPS based on matched results after match_store.py

introduce on CRPS: https://datumorphism.leima.is/cards/time-series/crps/#fn:3

In [50]:
import numpy as np
import torch
import math
import properscoring as ps

## Corner-based format with single-variate Gaussian distribution
kl_loss_corner

In [51]:
def compute_crps_corner_single(data_path, ty = 0, info=None):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    crps_list = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        std = np.sqrt(np.exp(cov))
        if ty == 0:
            std = std
        elif ty == 1:
            std = np.multiply(std, np.array(info))
        elif ty == 2:
            std = std * info
        crps = ps.crps_gaussian(target, 
                               pred,
                               std)
        crps_list.extend(np.mean(crps, axis=1).tolist())
    return sum(crps_list) / len(crps_list)

### disco

In [52]:
data_path = "check/check_loss_two_step_corner/disco/no_rsu/match_all_data_100_5.npy"
compute_crps_corner_single(data_path, 0)

0.30963438578820995

In [53]:
# 68.27%
quantile = [5.685563231556715, 5.7333907000666064, 5.8156107496472655, 5.698934531905109, 5.349435233304022, 5.286866601436044, 5.364550922522339, 5.254597136256393]
compute_crps_corner_single(data_path, 1, quantile)

0.28778308365641114

In [54]:
# 99.73%
quantile = np.array([48.667429272195506, 47.12830691785341, 47.89193980424878, 42.536190890924544, 42.169949853660526, 46.371733470171094, 41.253330827583945, 41.55867648053199]) / 3
compute_crps_corner_single(data_path, 1, quantile)

0.29421634107209993

In [55]:
compute_crps_corner_single(data_path, 2, 10)

0.28693554856029735

### Upperbound

In [56]:
data_path = "check/check_loss_two_step_corner/upperbound/no_rsu/match_all_data_100_5.npy"

In [57]:
compute_crps_corner_single(data_path, 0)

0.45255348877285945

In [58]:
# 68.27%
quantile = [4.190345185940821, 4.351993435985297, 4.558479310459919, 4.717549387208282, 4.278715029642637, 4.102446480783583, 4.1725601258012475, 3.530299361843318]
compute_crps_corner_single(data_path, 1, quantile)

0.4332851795817319

In [59]:
# 99.73%
quantile = np.array([49.72913250209128, 68.07916849224074, 54.76645263747507, 76.2931006003976, 46.32874805418221, 76.94147233490142, 50.70841262977031, 77.77131089570797]) / 3
compute_crps_corner_single(data_path, 1, quantile)

0.45320859651701817

In [60]:
compute_crps_corner_single(data_path, 2, 10)

0.431471650320429

### lowerbound

In [61]:
data_path = "check/check_loss_two_step_corner/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_crps_corner_single(data_path, 0)

0.4656329747412102

In [62]:
# 68.27%
quantile = [6.10302951960435, 6.182331226887726, 5.740435199991636, 6.236356538776135, 6.334263986132204, 5.1828751359733305, 6.399557006521807, 5.073939114533967]
compute_crps_corner_single(data_path, 1, quantile)

0.4426874117855666

In [63]:
# 99.73%
quantile = np.array([259.92825137969436, 201.07436055964888, 258.2222934053669, 197.05243747520606, 328.5189148613409, 235.0538877505854, 339.6938946129562, 235.97804342045072]) / 3
compute_crps_corner_single(data_path, 1, quantile)

0.6594356992282471

In [64]:
compute_crps_corner_single(data_path, 2, 10)

0.44084214608819644

## Center-based format with single-variate Gaussian distribution
kl_loss_center_ind

In [65]:
def corner_to_center_box2d_torch(corners):
    p1 = corners[:,0:2]
    p2 = corners[:,2:4]
    p3 = corners[:,4:6]
    p4 = corners[:,6:8]
    center = (p1+p2+p3+p4)/4
    w = (np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p2-p3, 2), axis=1, keepdims = True)))/2
    h = (np.sqrt(np.sum(np.power(p1-p2, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p3-p4, 2), axis=1, keepdims = True)))/2
    wp = np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True))
    sina = np.divide((p4[:,1:2] - p1[:,1:2]), wp)
    cosa = np.divide((p1[:,0:1] - p4[:,0:1]), wp)
    result = np.concatenate((center, w, h, sina, cosa), axis=1)
    return result

def compute_crps_center_single(data_path, ty = 0, info=None):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    crps_list = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = corner_to_center_box2d_torch(prediction[:,:8])
        target = corner_to_center_box2d_torch(np.array(tp[i][1]))
        cov = prediction[:,9:]
        std = np.sqrt(np.exp(cov))
        if ty == 0:
            std = std
        elif ty == 1:
            std = np.multiply(std, np.array(info))
        elif ty == 2:
            std = std * info
        crps = ps.crps_gaussian(target, 
                               pred,
                               std)
        crps_list.extend(np.mean(crps, axis=1).tolist())
    return sum(crps_list) / len(crps_list)

### disco

In [66]:
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 0)

0.09476382565762675

In [67]:
# 68.27%
quantile = [5.468984395243384, 4.89869516767041, 16.09847077479864, 3.0882512529684347, 1.0243907317028713, 4.811753752332904]
compute_crps_center_single(data_path, 1, quantile)

0.08243371697173087

In [68]:
# 99.73%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([48.40839894811819, 25.026924203602636, 131.90336169825727, 15.255118427560566, 59.67641819394204, 71.02182939111708]) / 3
compute_crps_center_single(data_path, 1, quantile)

0.09451582780211552

In [69]:
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 2, 10)

0.0904882054152939

In [70]:
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 2, 100)

0.3488100266123857

### upperbound

In [71]:
data_path = "check/check_loss_two_step_center_ind/upperbound/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 0)

0.11279209221508091

In [72]:
# 68.27%
data_path = "check/check_loss_two_step_center_ind/upperbound/no_rsu/match_all_data_100_5.npy"
quantile = [3.9270005592398936, 3.7293620083399235, 12.661970846621758, 3.507125215592104, 1.3240707560700242, 3.309770337192223]
compute_crps_center_single(data_path, 1, quantile)

0.10140134104554115

In [73]:
# 99.73%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([31.340185602346363, 24.276376908593676, 118.64887029003187, 18.253235606033712, 109.68835650542137, 56.51205738298187]) / 3
compute_crps_center_single(data_path, 1, quantile)

0.09328114061250656

In [74]:
data_path = "check/check_loss_two_step_center_ind/upperbound/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 2, 10)

0.10919984219658703

In [75]:
compute_crps_center_single(data_path, 2, 100)

0.3974998196751098

### lowerbound

In [76]:
data_path = "check/check_loss_two_step_center_ind/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 0)

0.12935502133502605

In [77]:
# 68.27%
data_path = "check/check_loss_two_step_center_ind/lowerbound/no_rsu/match_all_data_100_5.npy"
quantile = [4.824761883365095, 4.435752094000069, 14.379824436595687, 2.8108565130701026, 1.0106651376400175, 4.404153510782447]
compute_crps_center_single(data_path, 1, quantile)

0.11601606398440002

In [78]:
# 99.73%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([44.646381707137465, 28.842831596825288, 127.50113292712857, 18.538402799100826, 81.66889262783303, 53.23585004452376]) / 3
compute_crps_center_single(data_path, 1, quantile)

0.09481660538899857

In [79]:
data_path = "check/check_loss_two_step_center_ind/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_crps_center_single(data_path, 2, 10)

0.12334249409213853

In [80]:
compute_crps_center_single(data_path, 2, 100)

0.3886493278938143